## Imports & Sparksession init

In [1]:
# Importing packages
import os
import sys
import string
import re

# Installing pre-reqs
# !pip install twython

# Importing nltk-related junk
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.sentiment import SentimentIntensityAnalyzer

# Importing pyspark tools & init sparksession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, when
from pyspark.sql.types import IntegerType, StringType, FloatType, NumericType
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF
from pyspark.ml.classification import NaiveBayes, LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark = SparkSession.builder.master("local[4]")\
                    .config("spark.executor.memory", "8g")\
                    .config("spark.driver.memory", "8g")\
                    .config("spark.network.timeout", "10000000")\
                    .config("spark.executor.heartbeatInterval", "10000000")\
                    .getOrCreate()

# Checking spark session status
spark

# Importing training/testing set
train = spark.read.format("csv").load("train.csv", header=True)
test = spark.read.format("csv").load("test.csv", header=True)

# Subsetting from training data for quick dev testing
df = train.limit(20)
df.printSchema() 

############ DEFINING UDF HELPER FUNCTIONS FOR BULK CLEANING ############

# WORKFLOW STEP 1:
### remove non ASCII characters
def strip_non_ascii(data_str):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in data_str if 0 < ord(c) < 127)
    return ''.join(stripped)

# WORKFLOW STEP 2
### fixed abbreviation
def fix_abbreviation(data_str):
    data_str = data_str.lower()
    data_str = re.sub(r'\bthats\b', 'that is', data_str)
    data_str = re.sub(r'\bive\b', 'i have', data_str)
    data_str = re.sub(r'\bim\b', 'i am', data_str)
    data_str = re.sub(r'\bya\b', 'yeah', data_str)
    data_str = re.sub(r'\bcant\b', 'can not', data_str)
    data_str = re.sub(r'\bdont\b', 'do not', data_str)
    data_str = re.sub(r'\bwont\b', 'will not', data_str)
    data_str = re.sub(r'\bid\b', 'i would', data_str)
    data_str = re.sub(r'wtf', 'what the fuck', data_str)
    data_str = re.sub(r'\bwth\b', 'what the hell', data_str)
    data_str = re.sub(r'\br\b', 'are', data_str)
    data_str = re.sub(r'\bu\b', 'you', data_str)
    data_str = re.sub(r'\bk\b', 'OK', data_str)
    data_str = re.sub(r'\bsux\b', 'sucks', data_str)
    data_str = re.sub(r'\bno+\b', 'no', data_str)
    data_str = re.sub(r'\bcoo+\b', 'cool', data_str)
    data_str = re.sub(r'rt\b', '', data_str)
    data_str = data_str.strip()
    return data_str

# WORKFLOW STEP 3:
### remove irrelevant features
def remove_features(data_str):
    # compile regex
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    num_re = re.compile('(\\d+)')
    mention_re = re.compile('@(\w+)')
    alpha_num_re = re.compile("^[a-z0-9_.]+$")
    # convert to lowercase
    data_str = data_str.lower()
    # remove hyperlinks
    data_str = url_re.sub(' ', data_str)
    # remove @mentions
    data_str = mention_re.sub(' ', data_str)
    # remove punctuation
    data_str = punc_re.sub(' ', data_str)
    # remove numeric 'words'
    data_str = num_re.sub(' ', data_str)
    # remove non a-z 0-9 characters and words shorter than 1 characters
    list_pos = 0
    cleaned_str = ''
    for word in data_str.split():
        if list_pos == 0:
            if alpha_num_re.match(word) and len(word) > 1:
                cleaned_str = word
            else:
                cleaned_str = ' '
        else:
            if alpha_num_re.match(word) and len(word) > 1:
                cleaned_str = cleaned_str + ' ' + word
            else:
                cleaned_str += ' '
        list_pos += 1
    # remove unwanted space, *.split() will automatically split on
    # whitespace and discard duplicates, the " ".join() joins the
    # resulting list into one string.
    return " ".join(cleaned_str.split())

# pyspark udf functions

udf_ascii_strip = udf(strip_non_ascii, StringType())
udf_fix_abbrevs = udf(fix_abbreviation, StringType())
udf_rm_features = udf(remove_features, StringType())

root
 |-- textID: string (nullable = true)
 |-- text: string (nullable = true)
 |-- selected_text: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [2]:
############ CLEANING STEPS ON SMALL DF ############

# Calling helper UDFs on dev short data.frame
df = df.withColumn('stage01',udf_ascii_strip(df['selected_text']))
df = df.withColumn('stage02',udf_fix_abbrevs(df['stage01']))
df = df.withColumn('cleaned',udf_rm_features(df['stage02']))

# NLTK's VADER implementation
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

# Defining NLTK sentiment pyfunction
def get_nltk_sentiment(txt):
  tmp = sia.polarity_scores(txt)
  return(tmp['compound'])

# pyspark udf function for nltk
sentiment_analysis_udf = udf(get_nltk_sentiment, FloatType()) 

# Applying udf and print results
df.select('textID','cleaned','sentiment')\
    .withColumn("score_nltk", sentiment_analysis_udf( df['cleaned'] ))\
    .show()

# Tokenizer + Stop-word remover + Hash vectorizer
tokenizer =        Tokenizer( inputCol="cleaned",  outputCol="words" )
remover   = StopWordsRemover( inputCol="words",    outputCol="filtered" )
hasher    =        HashingTF( inputCol="filtered", outputCol="features" )


# Applying created objects to perform tokenizing, stop word removal, and hash vectorizer
df = tokenizer.transform(df)
df = remover.transform(df)
df = hasher.transform(df)

# Selecting only necessary columns
df = df.select('textID','sentiment','filtered','features')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


+----------+--------------------+---------+----------+
|    textID|             cleaned|sentiment|score_nltk|
+----------+--------------------+---------+----------+
|cb774db0d1|have responded if...|  neutral|       0.0|
|549e992a42|            sooo sad| negative|   -0.4767|
|088c60f138|         bullying me| negative|   -0.5994|
|9642c003ef|      leave me alone| negative|    -0.296|
|358bd9e861|             sons of| negative|       0.0|
|28b57f3990|some shameless pl...|  neutral|    0.4215|
|6e0c6d75b1|                 fun| positive|    0.5106|
|50e14c0bb8|          soooo high|  neutral|       0.0|
|e050245fbd|         both of you|  neutral|       0.0|
|fc2cbefa9d|wow you just beca...| positive|    0.5859|
|2339a9b08b|as much as love t...|  neutral|    0.8519|
|16fab9f95b|                like| positive|    0.3612|
|74a76f6e0a|         dangerously| negative|   -0.4588|
|04dd1d2e34|                lost| negative|   -0.3182|
|bbe3cbf620|test test from th...|  neutral|       0.0|
|8a939bfb5

In [3]:
############ CLEANING STEPS ON train0 ############

tokenizer =        Tokenizer( inputCol="cleaned",  outputCol="words" )
remover   = StopWordsRemover( inputCol="words",    outputCol="filtered" )
hasher    =        HashingTF( inputCol="filtered", outputCol="features" )

# Calling helper udfs on dev short data.frame
train0 =  train.withColumn('stage01',udf_ascii_strip( train['selected_text']))
train0 = train0.withColumn('stage02',udf_fix_abbrevs(train0['stage01']))
train0 = train0.withColumn('cleaned',udf_rm_features(train0['stage02']))
train0 = train0.select('textID','sentiment','cleaned')

# Applying created objects to perform tokenizing and stop word removal
train0 = tokenizer.transform(train0)
train0 = remover.transform(train0)

# Applying token hash vectorizer
train0   = hasher.transform(train0)

# Selecting only necessary columns
train0 = train0.select('textID','sentiment','filtered','features')

In [4]:
############ CLEANING STEPS ON test0 ############

tokenizer   =        Tokenizer( inputCol="cleaned",  outputCol="words")
remover     = StopWordsRemover( inputCol="words",    outputCol="filtered")

# Calling helper udfs on dev short data.frame
test0 =  test.withColumn('stage01',udf_ascii_strip(test['text']))
test0 = test0.withColumn('stage02',udf_fix_abbrevs(test0['stage01']))
test0 = test0.withColumn('cleaned',udf_rm_features(test0['stage02']))

# Applying created objects to perform tokenizing and stop word removal
test0 = tokenizer.transform(test0)
test0 = remover.transform(test0)

# Applying token count vectorizer
test0 = hasher.transform(test0)

# Selecting only necessary columns
test0 = test0.select('textID','sentiment','filtered','features')

In [5]:
# Transforming target variable to numeric type (neutral = 1, negative = 0, positive = 2)
train0 = train0.withColumn("sentiment", when(train0.sentiment == "neutral", "1")\
                                       .when(train0.sentiment == "negative", "0")\
                                       .when(train0.sentiment == "positive", "2"))
test0  = test0.withColumn("sentiment",  when(test0.sentiment == "neutral", "1")\
                                       .when(test0.sentiment == "negative", "0")\
                                       .when(test0.sentiment == "positive", "2"))

train0 = train0.select('textID','filtered','features',col("sentiment").cast('int'))
test0  = test0.select('textID','filtered','features',col("sentiment" ).cast('int'))

In [6]:
train0.show()
test0.show()

+----------+--------------------+--------------------+---------+
|    textID|            filtered|            features|sentiment|
+----------+--------------------+--------------------+---------+
|cb774db0d1|  [responded, going]|(262144,[102382,1...|        1|
|549e992a42|         [sooo, sad]|(262144,[125638,1...|        0|
|088c60f138|          [bullying]|(262144,[122399],...|        0|
|9642c003ef|      [leave, alone]|(262144,[27308,23...|        0|
|358bd9e861|              [sons]|(262144,[190202],...|        0|
|28b57f3990|[shameless, plugg...|(262144,[91665,11...|        1|
|6e0c6d75b1|               [fun]|(262144,[23087],[...|        2|
|50e14c0bb8|       [soooo, high]|(262144,[136020,1...|        1|
|e050245fbd|                  []|      (262144,[],[])|        1|
|fc2cbefa9d|[wow, became, coo...|(262144,[4631,119...|        2|
|2339a9b08b|[much, love, hope...|(262144,[13007,43...|        1|
|16fab9f95b|              [like]|(262144,[208258],...|        2|
|74a76f6e0a|       [dange

In [7]:
# Fitting Naive-Bayes
nb = NaiveBayes(modelType="multinomial",labelCol="sentiment", featuresCol="features")
nbModel = nb.fit(train0)
nb_predictions = nbModel.transform(test0)

## Evaluating Naive-Bayes
evaluator = MulticlassClassificationEvaluator(labelCol="sentiment", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))

Py4JJavaError: An error occurred while calling o389.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6) (bird-g706.attlocal.net executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.flushBuffer(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:307)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(PythonRDD.sc

In [8]:
## Fitting the model
lr = LogisticRegression(featuresCol = 'features', labelCol = 'sentiment', maxIter=10)
lrModel = lr.fit(train0)
lrPreds = lrModel.transform(test0)

## Evaluating the model
evaluator = MulticlassClassificationEvaluator(labelCol="sentiment", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lrPreds)
print("Accuracy of Logistic Regression is = %g"% (lr_accuracy))

Py4JJavaError: An error occurred while calling o418.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 7) (bird-g706.attlocal.net executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.flushBuffer(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:307)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(PythonRDD.sc

In [9]:
## Fitting the model
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'sentiment')
rfModel = rf.fit(train0)
rfPreds = rfModel.transform(test0)

## Evaluating the model
evaluator = MulticlassClassificationEvaluator(labelCol="sentiment", predictionCol="prediction", metricName="accuracy")
rf_accuracy = evaluator.evaluate(rfPreds)
print("Accuracy of Random Forests is = %g"% (rf_accuracy))

Py4JJavaError: An error occurred while calling o463.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 11) (bird-g706.attlocal.net executor driver): java.net.SocketException: Connection reset by peer: socket write error
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(Unknown Source)
	at java.net.SocketOutputStream.write(Unknown Source)
	at java.io.BufferedOutputStream.flushBuffer(Unknown Source)
	at java.io.BufferedOutputStream.write(Unknown Source)
	at java.io.DataOutputStream.write(Unknown Source)
	at java.io.FilterOutputStream.write(Unknown Source)
	at org.apache.spark.api.python.PythonRDD$.write$1(PythonRDD.scala:295)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1(PythonRDD.scala:307)
	at org.apache.spark.api.python.PythonRDD$.$anonfun$writeIteratorToStream$1$adapted(PythonRDD